# Predict Housing Prices

Iremos trabalhar com os dados do [Housing Prices Competition for Kaggle Users](https://www.kaggle.com/c/home-data-for-ml-course). 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Esse dataset contêm 79 variáveis explanatórias que descrevem praticamente todos os aspectos de uma casa. O intuito do programa é prever o preço das casas em Iowa, para que dessa forma o usuário consiga saber se um certo imóvel está com um preço muito elevado ou então se está com um preço baixo demais, o que poderia levar a uma corretora comprar esses imóveis para revendê-los futuramente. Também alguém interessado em vender sua residência poderia utilizar o código para saber por qual valor ele ou ela deveria vender, entre outras diversas funcionalidades.  

In [9]:
!pip install xgboost

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Primeiro começarei importando as Bibliotecas que serão utilizadas

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

## importar Dados

In [33]:
# Read the data
X = pd.read_csv(r'C:\Users\Marcelo\Downloads\train.csv', index_col='Id')
X_test_full = pd.read_csv(r'C:\Users\Marcelo\Downloads\test.csv', index_col='Id')

Uma vez que os dados foram importandos em um arquivo csv sem nenhum problema, podemos começar a análise

## Análise Exploratória

In [34]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [35]:
X.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

Com essa exploração, conseguimos ter uma noção melhor do que se trata os features, ou seja, as colunas desse dataset, como por exemplo: ano que a casa foi construída, rua em que a casa está localizada, tamanho do lote, etc

In [36]:
X['Neighborhood'].unique()

array(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste'], dtype=object)

In [37]:
X.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [38]:
X.shape

(1460, 80)

In [39]:
cols_with_nan = X.isnull().sum()
print(cols_with_nan[cols_with_nan>0])

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


Podemos observar que nesse dataset temos muitos missing values, ou seja, dados faltantes. Esse problema é muito comum quando se trata de dados do cotidiano e temos que resolvê-lo, pois a maioria dos modelos de Machine Learning apresenta um erro quando se tenta construir um modelo com missing values. Dessa forma, temos as seguintes opções para lidar com esse problema: 

* **Excluir Colunas com Missing Values**

A menos que a maior parte dos dados estejam faltando, como é no caso das colunas: "PoolQC", "Fence", "Alley" e "MiscFeature", 
excluir toda a coluna faz com que o modelo perca acesso a muita informação potencialmente útil. Dessa forma, esse medida se faz bem extrema e deve ser utiizada apenas quando quase todos os dados da coluna estiverem faltando ou quando a coluna não conter nenhuma informação que poderia ajudar com o modelo. 

* **Imputação (Label Encoder)**

Nesse caso, optamos por preencher os dados faltantes. As maneiras mais comuns de fazer esse preenchimento é com o valor médio da coluna ou os valores ou caracteres que mais se repetem. 

* **Uma Extensão à Imputação (One Hot Encoding)**

Já nesse outro caso, preenche-se os dados faltantes e se acrescenta novas colunas para dizer com valores binários onde houve ou não houve a imputação. Esse modelo se faz útil quando não se tem uma ordem categórica, ou seja, um char não tem maior valor que outro. E esse modelo não funciona muito bem quando se tem uma cardinalidade muito grande na coluna, ou seja, ela apresenta diversos objetos diferentes


## Antes de lidar com os Missing Values, primeiro prepararei os dados

In [40]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

Agora temos os dados separados em dados de treino e dados de validação. Esse medida se faz necessária para que não haja overfitting do modelo, ou seja, para que o modelo se adeque tão bem aos dados de treino, que não consiga fazer boas predições com outros dados que ainda não lhe foi apresentado. Então utilizamos a função train_test_split, assim podemos separar os dados de treino para o modelo.fit e os dados de validação para o modelo.predict

In [41]:
# "Cardinalidade" significa o número de valores únicos em uma coluna
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols

X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

Aqui, primeiro selecionei as colunas com baixa cardinalidade, esse valor é arbitrário, escolhi 10 para ser o número máximo, pois cada objeto único irá criar uma nova coluna binária, ou seja, se a cardinalidade é 10, terão 10 novas colunas para dizer onde foi aplicado o One-hot Encode.
Depois selecionei apenas as colunas numéricas do dataset e por fim juntei as colunas numéricas com as de baixa cardinalidade e filtrei meus dados de treino, validação e teste para terem somente essas colunas selecionadas. 

## Lidando com missing values

In [44]:
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

Aqui, por praticidade e para simplificar o código, utilizei o pandas para aplicar o One-Hot Encode. Para saber mais como funciona essa biblioteca é só clicar [aqui](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.get_dummies.html)

## Dados após o tratamento

In [45]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,0,1,0,0,0,0,0,0,0,1
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,0,0,0,1,0,0,0,0,1,0
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,0,0,0,1,0,0,0,0,1,0
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,0,0,0,1,0,0,0,0,1,0


Agora, já podemos notar a diferença no dataset. Creio que a que mais chama atenção é a quantidade de colunas em relação aos dados originais, que continham 80 e agora são 227 colunas, por conta do One-Hot-Encoder

## Aplicando Modelo de Machine Learning

In [46]:
# Define the model
model = XGBRegressor(n_estimators = 800, learning_rate = 0.05, random_state = 0)

# Fit the model
model.fit(X_train, y_train, early_stopping_rounds = 20,
          eval_set = [(X_valid, y_valid)], verbose = False)

# Get predictions
predictions = model.predict(X_valid)

c:\users\marcelo\appdata\local\programs\python\python35\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:23:41] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


O modelo que escolhi foi o XGBRegressor (Extreme Gradient Boosting) que funciona como a [descida de gradiente](http://deeplearningbook.com.br/aprendizado-com-a-descida-do-gradiente/). O algorítimo funciona da seguinte maneira, na primeira instância temos apenas um ensemble, como se fosse apenas uma árvore, então esse modelo faz uma previsão, calcula o erro, treina novamente como na descida de gradiente e adiciona um novo modelo ao emsemble, sendo análago ao algorítimo RandomForest. O número de vezes que é adicionado um novo modelo ao emsemble é definido pelo parâmetro n_estimators, nesse caso setei como 800 o número de vezes que o ciclo se repete.

## Avaliação do modelo utilizando os dados de validação

In [58]:
# Calculate MAE
mae = mean_absolute_error(y_valid, predictions) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae)


Mean Absolute Error: 16016.045416845034
